In [1]:
from IPython.core.display import display, HTML
display(HTML("<style> .container { width: 100% } </style>"))

In [2]:
import pandas as pd

In [3]:
df = pd.read_json('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.json')

### Отсеим страны с населением меньше 15 000 000 человек

In [4]:
df_filtered = df.T[
    (df.T.continent.apply(lambda continent: isinstance(continent, str)))
    & (df.T.population > 15_000_000)
].T

In [10]:
df_filtered.head(5)

,AFG,DZA,AGO,ARG,AUS,BGD,BRA,BFA,KHM,CMR,...,UGA,UKR,GBR,USA,UZB,VEN,VNM,YEM,ZMB,ZWE
continent,Asia,Africa,Africa,South America,Oceania,Asia,South America,Africa,Asia,Africa,...,Africa,Europe,Europe,North America,Asia,South America,Asia,Asia,Africa,Africa
location,Afghanistan,Algeria,Angola,Argentina,Australia,Bangladesh,Brazil,Burkina Faso,Cambodia,Cameroon,...,Uganda,Ukraine,United Kingdom,United States,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
population,39835428.0,44616626.0,33933611.0,45605823.0,25788217.0,166303494.0,213993441.0,21497097.0,16946446.0,27224262.0,...,47123533.0,43466822.0,68207114.0,332915074.0,33935765.0,28704947.0,98168829.0,30490639.0,18920657.0,15092171.0
population_density,54.422,17.348,23.89,16.177,3.202,1265.036,25.04,70.151,90.672,50.885,...,213.759,77.39,272.898,35.608,76.134,36.253,308.127,53.508,22.995,42.729
median_age,18.6,29.1,16.8,31.9,37.9,27.5,33.5,17.6,25.6,18.8,...,16.4,41.4,40.8,38.3,28.2,29.0,32.6,20.3,17.7,19.6


### Проитерируемся по df_filtered

In [20]:
for country in df_filtered:
    data = pd.DataFrame.from_dict(df_filtered[country].data)

### Новые колонки в датафрейме

In [21]:
column = 'new_cases'
minus = column + '_shift_minus'
plus = column + '_shift_plus'
data.loc[:, minus] = data[column].shift(-1)
data.loc[:, plus] = data[column].shift(1)

In [26]:
diff = 3
data = data.loc[(data['new_cases'].ge(100))
                & (((data[minus] - data[column]).abs() < diff) | ((data[plus] - data[column]).abs() < diff))
               ]

In [27]:
series_days_length = data.shape[0]